# The synApps `sscan`

The synApps `sscan` record is used to measure scans of detector(s) *v*. positioner(s).  It can be configured for step (linear) scans, list (table) scans, or fly scans.

**Goals**: Demonstrate use of the `sscan` record with Bluesky.

1. Press SCAN button of a preconfigured scan.

    1. Use a polling loop to wait for scan to end.
    2. Use a Status object to wait for scan to end.

2. Same, but connect with most `sscan` record fields and get data after scan.
3. Setup the same scan from bluesky.
4. Add the scan data as a bluesky run.

This notebook is intended for those who are familiar with EPICS and its motor, scaler, and sscan records but are new to Bluesky.

## Python setup

The first example will need only the bluesky RunEngine.  It will not need any
databroker catalog.  The EPICS IOC has a prefix of `gp:`.

In [1]:
import bluesky
import bluesky.plan_stubs as bps

RE = bluesky.RunEngine()
IOC = "gp:"

## Example 1 -- Press SCAN button of a preconfigured scan

Consider the case where the `sscan` record is already configured with all the
parameters necessary to perform a complete step scan.

The next screen (left) shows `gp:scan1` is configured to step scan motor `m1`
from -1.2 to 1.2 in 21 points, collecting counts from scaler `gp:scaler1`
channels 2 & 4 (`I0` & `diode`, respectively). The next screen (right) shows
`gp:scaler1` is configured with a counting time of 0.2 seconds per point and
several detector channels.

scaler | scan
--- | ---
![scaler1 setup](./scaler16.png) | ![scan1 setup](./sscan-scaler-v-motor.png)

The `sscan` is preconfigured and will begin running when the *SCAN* button is
pressed.  When the *SCAN* button is pressed, a `1` is sent to EPICS PV
`gp:scan1.EXSC` and the `sscan` starts.  When the `sscan` finishes, the value of
the button returns to `0`.

First, connect with the EPICS PV of the button using `ophyd.EpicsSignal`.  Once connected, show the current value.

In [2]:
from ophyd import EpicsSignal

scan_button = EpicsSignal(f"{IOC}scan1.EXSC", name="scan_button")
scan_button.wait_for_connection()
print(f"{scan_button.get()}")

0


Write a bluesky plan that pushes the button, waits a brief moment for the scan
to start, then waits for the scan to end.  This simple example waits by
periodically polling the value of the button.

Here, `scan_button.get(use_monitor=False)` forces ophyd to get a new value
from EPICS (and not to rely on the monitor value being updated promptly).
Probably not necessary to use `use_monitor=False`.

In [3]:
def run_sscan():
    yield from bps.mv(scan_button, 1)
    print("Started scan ...")
    yield from bps.sleep(0.1)  # wait for scan to start
    while scan_button.get(use_monitor=False) != 0:
        yield from bps.sleep(0.1)
    print("Scan done.")

Execute the `run_sscan()` plan using the bluesky RunEngine `RE`.

In [4]:
RE(run_sscan())

Started scan ...
Scan done.


()

To get the data from the `sscan` record, we use `ophyd.EpicsSignalRO` to connect with the arrays.  We'll do that in a later section of this notebook.

Let's eliminate that polling loop.  This will make the plan respond more quickly
when the button changes value.  The `ophyd.status.Status` class (runs in the
background) reports when the scan has finished or times out.  We define an inner
`watch_the_button()` function to detect when the button goes from `1` to `0`
(the scan stops).  We setup a subscription (*after* we start the move) to
monitor the button value and respond immediately.  We wait for the status object
to finish with `st.wait()`. Finally, we cancel the monitor of the button.

In [5]:
from ophyd.status import Status

def run_sscan():
    st = Status(timeout=20)

    def watch_the_button(old_value, value, **kwargs):
        if old_value == 1 and value == 0:
            st.set_finished()

    yield from bps.mv(scan_button, 1)
    scan_button.subscribe(watch_the_button)
    st.wait()
    scan_button.unsubscribe_all()

In [6]:
RE(run_sscan())

()

## Example 2 -- Run preconfigured `sscan` record

The `apstools.synApps.SscanRecord` class provides access to most of the fields
provided by a `sscan` record.  Use `SscanRecord` to connect and repeat the above
example.  With the `SscanRecord` class, the button is called `execute_scan`.

In [7]:
from apstools.synApps import SscanRecord

scan1 = SscanRecord(f"{IOC}scan1", name="scan1")
scan1.wait_for_connection()

def run_sscan():
    st = Status(timeout=20)

    def watch_the_button(old_value, value, **kwargs):
        if old_value == 1 and value == 0:
            st.set_finished()

    yield from bps.mv(scan1.execute_scan, 1)
    scan1.execute_scan.subscribe(watch_the_button)
    st.wait()
    scan1.execute_scan.unsubscribe_all()

In [8]:
RE(run_sscan())

()

Retrieve the data collected by `scan1` as a dictionary of numpy arrays.  Include
both detectors and the motor.  The `sscan` record has buffers capable of
collecting as many as 1,000 points per array.  First get the number of points
collected, then limit each array length to that number.

If we write this as a function, we can call it again later.  Since it executes quickly, it does not need to be written as a bluesky plan.

In [9]:
def get_sscan_data():
    npts = scan1.current_point.get()
    data = {
        "m1": scan1.positioners.p1.array.get()[:npts],
        "I0": scan1.detectors.d01.array.get()[:npts],
        "diode": scan1.detectors.d02.array.get()[:npts],
    }
    return data

get_sscan_data()

{'m1': array([-1.2 , -1.08, -0.96, -0.84, -0.72, -0.6 , -0.48, -0.36, -0.24,
        -0.12,  0.  ,  0.12,  0.24,  0.36,  0.48,  0.6 ,  0.72,  0.84,
         0.96,  1.08,  1.2 ]),
 'I0': array([1., 1., 0., 1., 1., 1., 0., 2., 1., 3., 4., 4., 3., 2., 1., 1., 1.,
        0., 2., 1., 0.], dtype=float32),
 'diode': array([0., 1., 1., 1., 1., 1., 1., 1., 2., 2., 4., 5., 3., 1., 2., 1., 1.,
        1., 1., 2., 1.], dtype=float32)}

## Example 3 -- Setup and run same scan from bluesky

Repeat the scan from the previous examples, but use bluesky to configure
`scan1`.  It will be useful to connect the motor, the scaler, and two the scaler
channels.

In [10]:
from ophyd import EpicsMotor
from ophyd.scaler import ScalerCH

m1 = EpicsMotor(f"{IOC}m1", name="m1")
scaler1 = ScalerCH(f"{IOC}scaler1", name="scaler1")
m1.wait_for_connection()
scaler1.wait_for_connection()

# for convenience
I0 = scaler1.channels.chan02.s
diode = scaler1.channels.chan04.s

We can supply the count time per point and scan range parameters as arguments to
the setup.  After setting the counting time for the scaler, the next step in the
setup is to clear any existing configuration of `scan1` using its `reset()`
method.  In a bluesky plan, we'll need to use
`apstools.plans.run_blocking_function` with that `reset()` method.  Finally,
we'll setup `scan1` with the EPICS PV names to be used.

In [11]:
from apstools.plans import run_blocking_function

def setup_scan1(start, finish, npts, ct=1):
    yield from bps.mv(scaler1.preset_time, ct)  # counting time/point
    yield from run_blocking_function(scan1.reset)
    yield from bps.sleep(0.2)  # arbitrary wait for EPICS record processing

    # positioners
    yield from bps.mv(
        scan1.number_points, npts,
        scan1.positioners.p1.start, start,
        scan1.positioners.p1.end, finish,
        scan1.positioners.p1.readback_pv, m1.user_readback.pvname,
        scan1.positioners.p1.setpoint_pv, m1.user_setpoint.pvname,
    )
    # triggers (push scaler count button at each point)
    yield from bps.mv(
        scan1.triggers.t1.trigger_pv, scaler1.count.pvname,
    )
    # detectors
    yield from bps.mv(
        scan1.detectors.d01.input_pv, I0.pvname,
        scan1.detectors.d02.input_pv, diode.pvname,
    )

Setup the scan.

In [12]:
RE(setup_scan1(-1.1, 1.1, 11, 0.2))

()

Run the scan.  We should not have to reprogram this plan.

In [13]:
RE(run_sscan())

()

Get the scan data.  Same function as before.  It's not a bluesky plan, so `RE()`
is not needed.

In [14]:
get_sscan_data()

{'m1': array([-1.1 , -0.88, -0.66, -0.44, -0.22,  0.  ,  0.22,  0.44,  0.66,
         0.88,  1.1 ]),
 'I0': array([1., 0., 2., 2., 1., 2., 2., 2., 1., 1., 2.], dtype=float32),
 'diode': array([1., 2., 1., 1., 2., 3., 2., 1., 2., 1., 1.], dtype=float32)}

## Post data to bluesky as a run

So far, we have just run `scan1` and shown the data collected.  We'll need to do
a bit more work to get this data into bluesky as a run.  While we might just use
`bps.read(scan1)` to get the data, the data would be named according to the
`scan1` structure.  We want more meaningful names so we re-assign the names of
the original objects (motor and scaler channel names).  To post *bluesky data*,
it must come from an ophyd `Device` (subclass).  We'll create a custom one just
for our setup of `scan1`, then `bps.mv()` each of the arrays.  Since we do not
have timestamps for each of the data points, we'll post the entire array as a
single event.  The event will have the timestamp from the `bps.mv()` plan stub.

The last five steps are all descriptive.  The run is opened, the `primary`
stream is written with the `scan_data`, then all is buttoned up and bluesky
finished the run.  We define `name="scan1"` so the names of the data in the databroker will match their usage here.  The `scan_data` object is only a tool to get the data from the `sscan` record into the databroker.

In [15]:
from ophyd import Component, Device, Signal

class ThisSscanData(Device):
    m1 = Component(Signal)
    I0 = Component(Signal)
    diode = Component(Signal)

scan_data = ThisSscanData("", name="scan1")

def post_sscan_data(md={}):
    data = get_sscan_data()
    yield from bps.mv(
        scan_data.m1, data["m1"],
        scan_data.I0, data["I0"],
        scan_data.diode, data["diode"],
    )
    yield from bps.open_run(md)
    yield from bps.create(name="primary")
    yield from bps.read(scan_data)
    yield from bps.save()
    yield from bps.close_run()

Now we need a databroker catalog.  Make a temporary one for this notebook.

In [16]:
import databroker

cat = databroker.temp().v2
RE.subscribe(cat.v1.insert)

0

Post the data from the recent scan as a new bluesky run.

Note: A bluesky plan can generate zero or more runs, each indexed by a `uid`.
Print the tuple of run uids returned from `RE()`.

In [17]:
uids = RE(post_sscan_data())
uids

('3e1451b1-255e-49ee-9a76-48f6f9708615',)

Get the run entry (by uid) from the catalog.

In [18]:
run = cat[uids[0]]
run

BlueskyRun
  uid='3e1451b1-255e-49ee-9a76-48f6f9708615'
  exit_status='success'
  2024-03-17 09:59:56.147 -- 2024-03-17 09:59:56.151
  Streams:
    * primary


Show the data arrays from the run's primary stream. The data are returned as a
single xarray [Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html).

In [19]:
run.primary.read()

<xarray.Dataset>
Dimensions:      (time: 1, dim_0: 11, dim_1: 11, dim_2: 11)
Coordinates:
  * time         (time) float64 1.711e+09
Dimensions without coordinates: dim_0, dim_1, dim_2
Data variables:
    scan1_m1     (time, dim_0) float64 -1.1 -0.88 -0.66 -0.44 ... 0.66 0.88 1.1
    scan1_I0     (time, dim_1) float32 1.0 0.0 2.0 2.0 1.0 ... 2.0 1.0 1.0 2.0
    scan1_diode  (time, dim_2) float32 1.0 2.0 1.0 1.0 2.0 ... 1.0 2.0 1.0 1.0

## Combined setup, scan, post data

Instead of calling the `RE()` separately with each of these plans, combine them
into an outer plan.

In [20]:
def combined(start, finish, npts, ct=1, md={}):
    yield from setup_scan1(start, finish, npts, ct=ct)
    yield from run_sscan()
    yield from post_sscan_data(md)

Run the combined plan and retrieve the data from the databroker catalog.

In [21]:
uids = RE(combined(-1.2, 1.2, 21, ct=.2))
cat[uids[0]].primary.read()

<xarray.Dataset>
Dimensions:      (time: 1, dim_0: 21, dim_1: 21, dim_2: 21)
Coordinates:
  * time         (time) float64 1.711e+09
Dimensions without coordinates: dim_0, dim_1, dim_2
Data variables:
    scan1_m1     (time, dim_0) float64 -1.2 -1.08 -0.96 -0.84 ... 0.96 1.08 1.2
    scan1_I0     (time, dim_1) float32 0.0 2.0 2.0 0.0 2.0 ... 1.0 0.0 1.0 1.0
    scan1_diode  (time, dim_2) float32 1.0 0.0 2.0 2.0 1.0 ... 0.0 0.0 0.0 1.0